In [843]:
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os
import pandas as pd
import numpy as np

In [844]:
#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
#The df
untouched_csv = pd.read_csv('data.csv', sep=',')

In [846]:
#replace the direction column with the corresponding degree values
direction_map = {'N': 0,'NNE': 22.5,'NE': 45,'ENE': 67.5,'E': 90,'ESE': 112.5,'SE': 135,'SSE': 157.5,
            'S': 180,'SSW': 202.5,'SW': 225,'WSW': 247.5,'W': 270,'WNW': 292.5,'NW': 315,'NNW': 337.5
        }

In [847]:
untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map)
#untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].astype(int)

C:\Users\Krupam\AppData\Local\Temp\ipykernel_9092\2514930122.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map)


In [848]:
#convert to Celsius
untouched_csv['Mainland Air Temp'] = round((untouched_csv['Mainland Air Temp']-32) * 5.0/9.0, 1)
untouched_csv['LBI Air Temp'] = round((untouched_csv['LBI Air Temp']-32) * 5.0/9.0, 1)


In [849]:
#rounds all colomns
untouched_csv['Humidity (%)'] = round(untouched_csv['Humidity (%)'], 1)
untouched_csv['Wind Speed (A)'] = round(untouched_csv['Wind Speed (A)'], 1)
untouched_csv['Gusting'] = round(untouched_csv['Gusting'], 1)
untouched_csv['Atmospheric Pressure (IN)'] = round(untouched_csv['Atmospheric Pressure (IN)'], 2)
untouched_csv['Precipitation Rate'] = round(untouched_csv['Precipitation Rate'], 2)
untouched_csv['Bay Temp'] = round(untouched_csv['Bay Temp'], 2)
untouched_csv['Salinity'] = round(untouched_csv['Salinity'], 2)
untouched_csv['LBI Air Temp'] = round(untouched_csv['LBI Air Temp'], 1)
untouched_csv['Ocean Temp'] = round(untouched_csv['Ocean Temp'], 1)

In [850]:
#determines if its a onshore breeze and adds a new column for it
onshore_degrees = [180, 135, 157.5, 90, 67.5, 45, 22.5]
untouched_csv['Onshore'] = untouched_csv['Direction (A)'].isin(onshore_degrees)
untouched_csv['Onshore'] = untouched_csv['Onshore'].astype(int)



In [851]:
#thresholds
ocean_thresh = 1.0
bay_thresh = 1.67
wind_thresh = 7.0
untouched_csv['rolling avg'] = untouched_csv['Bay Temp'].rolling(window=24).mean()

#Boolean math to check if it is upwelling or not
untouched_csv['bay_bool'] = (abs(untouched_csv['Bay Temp'] - untouched_csv['rolling avg']) > bay_thresh).astype(int)
untouched_csv['ocean_bool'] = (abs(untouched_csv["Bay Temp"] - untouched_csv["Ocean Temp"]) > ocean_thresh).astype(int)
untouched_csv['big_wind'] = (untouched_csv['Wind Speed (A)'] > wind_thresh).astype(int)
untouched_csv["upwelling_flag"] = ((untouched_csv['bay_bool'] == 1) & (untouched_csv['ocean_bool'] == 1) & (untouched_csv['big_wind'] == 1)).astype(int)
"""Explain this here"""

#Drop the temporary colomns
untouched_csv.drop(['bay_bool', 'ocean_bool', 'big_wind', 'rolling avg'], axis=1)
	
	
    

,Date,Hr,Mainland Air Temp,Humidity (%),Direction (A),Wind Speed (A),Gusting,Atmospheric Pressure (IN),Precipitation Rate,Bay Temp,Salinity,LBI Air Temp,Ocean Temp,Onshore,upwelling_flag
0,6/1/2025,0,11.8,88.0,270.0,6.8,12.1,29.54,0.0,17.52,28.34,13.4,14.5,0,0
1,6/1/2025,1,11.2,86.0,270.0,5.5,12.5,29.55,0.0,17.43,27.89,13.0,14.5,0,0
2,6/1/2025,2,10.4,83.0,247.5,4.5,9.3,29.57,0.0,17.42,27.90,12.8,14.4,0,0
3,6/1/2025,3,10.1,79.0,225.0,2.6,4.0,29.59,0.0,17.34,28.57,12.2,14.4,0,0
4,6/1/2025,4,8.7,86.0,247.5,3.9,6.2,29.60,0.0,17.23,28.20,12.2,14.4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,9/23/2025,20,21.2,88.0,0.0,6.0,9.1,29.96,0.0,21.84,30.30,22.4,21.3,0,0
2757,9/23/2025,21,20.9,89.0,202.5,4.2,6.4,29.96,0.0,21.61,29.96,22.6,21.3,0,0
2758,9/23/2025,22,21.4,83.0,202.5,4.5,7.5,29.96,0.0,22.02,30.06,23.0,21.3,0,0
2759,9/23/2025,23,21.5,84.0,225.0,3.8,5.3,29.97,0.0,22.09,30.03,22.9,21.3,0,0


In [ ]:
#Convert data into Date time and create date filter
untouched_csv['Date'] = pd.to_datetime(untouched_csv['Date'])

"""
Use this in future if data set needs specific dates
prediction = data.loc{
    (untouched_csv['Date'] > datetime(x, x, x)) &
    (untouched_csv['Date'] < datetime(x, x, x,))
}
"""

In [ ]:
#Save this data frame to a Finished csv

In [853]:
untouched_csv.head()

#info on csv
#print(untouched_csv.info())

#Number of upwelling events
"""
num_upwelling_events = untouched_csv['upwelling_flag'].sum()
print("Number of upwelling hours:", num_upwelling_events)
"""

#Find which rows where upwelling
"""
#upwelling_rows = untouched_csv[untouched_csv['upwelling_flag'] == 1]
#print(upwelling_rows)
"""

#print(untouched_csv.to_string())

"\n#upwelling_rows = untouched_csv[untouched_csv['upwelling_flag'] == 1]\n#print(upwelling_rows)\n"